#  **Project: Capstone Project 2– v5**

**Project name:** - Citi Bike– Repricing case study and Twitter Sentiment analysis for real time reputation management

**Student Name:** - Jitendra Agarwal

**Course:** - Springboard cohort Jan2 2018

**Summary:** - 	Citi Bike is the largest bike share program in us, with 10,000 bikes and 600 stations across 
Manhattan, Brooklyn, Queens and Jersey City. It was designed for quick trips with convenience in mind, and it’s a fun and affordable way to get around town. Everyone knows that bike sharing is the answer to many environmental and urban transportation issues, yet it’s not mainstream in US. 
I am being asked by the senior executive team at Citi bike to use data science techniques to recommend 3 key action item to increase the company’s’ business. 

**Problem Statement: - **
    • Citi bike management is curious to know if there is any statistical way to find the reason in decline in trips observed few times last quarter. 
    • How are user sentiments about a recent change in pricing and if Citi bike is really useful for users from time, cost and efficiency point of view compared to other transportation options? 
    • How is the performance of unit economics per trip or per bike or per station? o What is the most common use of Citi bike? 
    • What measure can be taken to increase user trips by 5% with existing users. 

**Project Goals: ** -
    • User Sentiment analysis from twitter and identify most common customer issues and how to address them. 
    • Analyze if any of the current plan can be repriced to get 5% increase on revenue with no customer impact. 
    • Can we predict a right number of bike to stationed as a station? o Propose a new monthly pass pricing for office commuters.

# ** Pre Processing and Data Wrangling

In [1]:
#Import Pandas and read csv into a DataFrame

import pandas as pd

def filter_csv_to_df(filename, filters, chunk):
    
    df_new=pd.DataFrame()

    if chunk <= 10000 or chunk >= 100000000:
        chunk = 10 ** 8
        
    for data in pd.read_csv(filename, chunksize=chunk, low_memory=False):
        df = pd.DataFrame(data)
        df_new = pd.concat([df_new, df], axis=0)
        
        if filters == 'N':
            filter_df = df_new.loc[df_new['usertype'] != 'Subscriber']
        else:
            filter_df = df_new.loc[df_new['usertype'] == 'Subscriber']
            
    return filter_df


In [2]:
import numpy as np
from numpy import math 
from math import sin, cos, sqrt, atan2

def mile_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6371 * c
    miles = km * 0.621371 * 1.414
    if miles > 5.5 :
        score = 0
    else :
        score = 1
    return score

In [3]:
# Get Weather Data and  Cleanup to get daily minimum and average temperature for each day of Q4.
# Get summary data provided by Citibike on no of subscriber and daily pass purchases.

df_LMT = pd.DataFrame(pd.read_csv('../data/LMT_Q42017.csv'))
df_AVGT = pd.DataFrame(pd.read_csv('../data/AVGT_Q42017.csv'))
df_summ = pd.DataFrame(pd.read_csv('../data/2017Q4-summary.csv'))

df1 = pd.DataFrame()
df1 = pd.concat([df1, df_LMT['Oct'], df_LMT['Nov'], df_LMT['Dec']], axis=0)
df1 = df1.rename(index=str, columns={0: 'LMT'})

df2 = pd.DataFrame()
df2 = pd.concat([df2, df_AVGT['Oct'], df_AVGT['Nov'], df_AVGT['Dec']], axis=0)
df2 = df2.rename(index=str, columns={0: 'AVGT'})

df= pd.concat([df1, df2], axis=1)
df.insert(0, 'id', range(1, 1 + len(df)))
df_temp = df.query('id != 62')
df_temp['startdate'] = pd.to_datetime(np.arange(92), unit='D', origin=pd.Timestamp('2017-10-01'))
df_temp.reset_index(inplace=True)
df_temp.set_index('startdate')

df_summ['startdate'] = pd.to_datetime(np.arange(92), unit='D', origin=pd.Timestamp('2017-10-01'))
df_temp['LMT'] = df_temp['LMT'].astype(int)
df_temp['AVGT'] = df_temp['AVGT'].astype(float)

print(df_summ.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 7 columns):
Date              92 non-null object
Trips             92 non-null int64
Miles             92 non-null int64
Members           92 non-null int64
24-Hour-Passes    92 non-null int64
3-Day-Passes      92 non-null int64
startdate         92 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 5.1+ KB
None


/Users/jitendra_agarwal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jitendra_agarwal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jitendra_agarwal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [4]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import bokeh.plotting as bkp
from mpl_toolkits.axes_grid1 import make_axes_locatable
sns.set()

file_list = ['../data/201710-citibike-tripdata.csv', '../data/201711-citibike-tripdata.csv', '../data/201712-citibike-tripdata.csv']

df_sub = pd.DataFrame()
df_cust = pd.DataFrame()

for file in file_list:
        data_sub = filter_csv_to_df(file, 'Y', 100000000)
        
        df_sub = pd.concat([df_sub, data_sub], axis=0)
        
        data_cust = filter_csv_to_df(file, 'N', 100000000)

        df_cust = pd.concat([df_cust, data_cust], axis=0)

        

In [5]:
print (df_sub.shape)
print (df_cust.shape)

(3789518, 15)
(328690, 15)


In [6]:
# Seperate DATES and TIME in standard format
# Convert birth year to int, calculate User age and drop User with age > 99 (outliers)

# Calculate Driver Age.
df_sub = df_sub.dropna(subset = ['birth year'])
df_cust = df_cust.dropna(subset = ['birth year'])

df_sub['birth year'] = df_sub['birth year'].astype(int)
df_cust['birth year'] = df_cust['birth year'].astype(int)

df_sub['driver_age'] = pd.to_datetime('today').year-df_sub['birth year']
df_cust['driver_age'] = pd.to_datetime('today').year-df_cust['birth year']


df_sub = df_sub.query("driver_age < 100 ")
df_cust = df_cust.query("driver_age < 100 ")

### Rename Few column for easy reference

df_sub = df_sub.rename(index=str, columns={"start station id": 'start_id', "start station name": "start_name"})
df_cust = df_cust.rename(index=str, columns={"start station id": "start_id", "start station name": "start_name"})

df_sub = df_sub.rename(index=str, columns={"end station id": "end_id", "end station name": "end_name"})
df_cust = df_cust.rename(index=str, columns={"end station id": "end_id", "end station name": "end_name"})

df_sub = df_sub.rename(index=str, columns={"start station latitude": "start_lat", "start station longitude": "start_lon"})
df_cust = df_cust.rename(index=str, columns={"start station latitude": "start_lat", "start station longitude": "start_lon"})

df_sub = df_sub.rename(index=str, columns={"end station latitude": "end_lat", "end station longitude": "end_lon"})
df_cust = df_cust.rename(index=str, columns={"end station latitude": "end_lat", "end station longitude": "end_lon"})

df_sub.insert(0, 'trip_id', range(1, 1 + len(df_sub)))
df_cust.insert(0, 'trip_id', range(1, 1 + len(df_cust)))


In [7]:
# Extract date and hour from Datetime column
# Create a unique trip id for each trip

df_sub['startdate'] = pd.to_datetime(df_sub['starttime']).dt.date
df_cust['startdate'] = pd.to_datetime(df_cust['starttime']).dt.date

df_sub['starthour'] = pd.to_datetime(df_sub['starttime']).dt.hour
df_cust['starthour'] = pd.to_datetime(df_cust['starttime']).dt.hour

df_sub['enddate'] = pd.to_datetime(df_sub['stoptime']).dt.date
df_cust['enddate'] = pd.to_datetime(df_cust['stoptime']).dt.date

df_sub['endhour'] = pd.to_datetime(df_sub['stoptime']).dt.hour
df_cust['endhour'] = pd.to_datetime(df_cust['stoptime']).dt.hour

df_sub['start_day'] = pd.to_datetime(df_sub['starttime']).dt.dayofweek
df_cust['start_day'] = pd.to_datetime(df_cust['starttime']).dt.dayofweek

df_sub['end_day'] = pd.to_datetime(df_sub['stoptime']).dt.dayofweek
df_cust['end_day'] = pd.to_datetime(df_cust['stoptime']).dt.dayofweek

In [8]:
# Create New columns to identify if Trip was free i.e covered in the pass or not
# what are paid units of each trip and $ paid based on plans

df_sub['free_trip'] = np.where(df_sub['tripduration'] <= 2700, 1, 0)
df_cust['free_trip'] = np.where(df_cust['tripduration'] <= 1800, 1, 0)

df_sub['paidunits'] = np.where(df_sub['free_trip'] == 0 ,np.ceil((df_sub['tripduration']-2700)/900), 0)
df_cust['paidunits'] = np.where(df_cust['free_trip'] == 0 ,np.ceil((df_cust['tripduration']-1800)/900), 0)

df_sub['paidamount'] = np.where(df_sub['paidunits'] > 0 ,df_sub['paidunits']*2.5, 0)
df_cust['paidamount'] = np.where(df_cust['paidunits'] > 0 ,df_cust['paidunits']*4.0, 0)

print (df_sub.shape)
print (df_cust.shape)

(3726266, 26)
(64431, 26)


In [9]:
# Create a DF for unique list of station

df_station1 = df_sub[['start_id', 'start_name', 'start_lat', 'start_lon']]
df_station2 = df_sub[['end_id', 'end_name', 'end_lat', 'end_lon']]

df_station1['start_id'] = df_sub['start_id'].astype(int)
df_station2['end_id'] = df_sub['end_id'].astype(int)

df_station1.drop_duplicates()
df_station2.drop_duplicates()

/Users/jitendra_agarwal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jitendra_agarwal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,end_id,end_name,end_lat,end_lon
0,478,11 Ave & W 41 St,40.760301,-73.998842
2,350,Clinton St & Grand St,40.715595,-73.987030
5,468,Broadway & W 56 St,40.765265,-73.981923
7,305,E 58 St & 3 Ave,40.760958,-73.967245
8,355,Bayard St & Baxter St,40.716021,-73.999744
9,295,Pike St & E Broadway,40.714067,-73.992939
10,3090,N 8 St & Driggs Ave,40.717746,-73.956001
12,458,11 Ave & W 27 St,40.751396,-74.005226
13,3346,Berkeley Pl & 7 Ave,40.675147,-73.975232
14,3314,W 95 St & Broadway,40.793770,-73.971888


In [10]:
# Create a Df for unique list of station
df_station1.append(df_station2)

df_stations = df_station1.rename(index=str, columns={"start_id": 'station_id', "start_name": "station_name", "start_lat": 'station_lat', "start_lon": "station_lon"})

df_stations.set_index(['station_id'], append=True)
df_stations = df_stations.sort_values(['station_id']).drop_duplicates(subset=['station_id'])

print(df_stations.info())

<class 'pandas.core.frame.DataFrame'>
Index: 772 entries, 26364 to 819070
Data columns (total 4 columns):
station_id      772 non-null int64
station_name    772 non-null object
station_lat     772 non-null float64
station_lon     772 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 30.2+ KB
None


In [11]:
df = []
near_df = []
k = len(df_stations)
for i in range(0, k):
    near_count = 0
    for j in range(0, k):
        near =  mile_distance(df_stations.iloc[i]['station_lon'], df_stations.iloc[i]['station_lat'], df_stations.iloc[j]['station_lon'], df_stations.iloc[j]['station_lat'])
        near_count  = near_count + near;
        near_df.append(near_count)
    df.append({'near_count': near_count, 'origin': df_stations.iloc[i]['station_id']})    
    df_near = pd.DataFrame(df)

df_near.head()


,near_count,origin
0,473,72
1,516,79
2,530,82
3,413,83
4,483,116


In [12]:
#Calculate Unit Economics and create Monthly summary and calculate total monthyl revenue based on pricing

trip_bymonth = df_sub.groupby(pd.to_datetime(df_sub['startdate']).dt.month).agg({'tripduration': 'sum', 'bikeid':'nunique', 'trip_id':'count', 'paidunits':'sum' })
trip_bymonth = trip_bymonth.rename(index=str, columns={"tripduration": 'total_trip_time', "bikeid": "no_of_bikes", "trip_id": "no_of_trips"})

trip_bymonth_cust = df_cust.groupby(pd.to_datetime(df_cust['startdate']).dt.month).agg({'tripduration': 'sum', 'bikeid':'nunique', 'start_id':'count', 'paidunits':'sum' })
trip_bymonth_cust = trip_bymonth_cust.rename(index=str, columns={"tripduration": 'total_trip_time', "bikeid": "no_of_bikes", "start_id": "no_of_trips"})

trip_bymonth_summ = df_summ.groupby(pd.to_datetime(df_summ['Date']).dt.month).agg({'Trips': 'sum', 'Miles':'sum', 'Members':'max', '24-Hour-Passes':'sum', '3-Day-Passes':'sum' })

print('--------------------------------------------------------------------------')

avg_mile=7.456/60/60
print('As per Citi Bike, avg distance traveled per second in miles is=',avg_mile ) 
print('Total unique bikes used by subscribers in Q4=',len(df_sub['bikeid'].unique() )) 
print('Total No of Trips by subscribers in Q4=',len(df_sub['bikeid']) )
print('Each Subscriber pays per month=', 14.95)
print('--------------------------------------------------------------------------')

#caclulate total mile traveled, average mile per bike, avg time per bike, average mile per trip, average time per trip
trip_bymonth['month'] = ['OCT', 'NOV', 'DEC']
trip_bymonth['total_distance'] = np.around(trip_bymonth['total_trip_time']*avg_mile, decimals=2)

trip_bymonth['avg_dis_per_trip'] = np.around(trip_bymonth['total_distance']/trip_bymonth['no_of_trips'], decimals=2)
trip_bymonth['avg_time_per_trip'] = np.around((trip_bymonth['total_trip_time']/trip_bymonth['no_of_trips'])/60, decimals=2)

trip_bymonth['avg_trip_per_bike'] = np.around(trip_bymonth['no_of_trips']/trip_bymonth['no_of_bikes'], decimals=2)
trip_bymonth['avg_dis_per_bike'] = np.around(trip_bymonth['total_distance']/trip_bymonth['no_of_bikes'], decimals=2)
trip_bymonth['avg_time_per_bike'] = np.around((trip_bymonth['total_trip_time']/trip_bymonth['no_of_bikes'])/60, decimals=2)

trip_bymonth_summ['pass_revenue_bysubs'] = np.around(trip_bymonth_summ['Members']*14.95, decimals=2)
trip_bymonth_summ['pass_revenue_bycust_1d'] = np.around(trip_bymonth_summ['24-Hour-Passes']*12, decimals=2)
trip_bymonth_summ['pass_revenue_bycust_3d'] = np.around(trip_bymonth_summ['3-Day-Passes']*24, decimals=2)

trip_bymonth['paid_trip_revenue_bysubs'] = np.around(trip_bymonth['paidunits']*2.5, decimals=2)
trip_bymonth_cust['paid_trip_revenue_bycust'] = np.around(trip_bymonth_cust['paidunits']*4, decimals=2)

trip_bymonth['paid_trip_monthly_revenue'] = trip_bymonth['paid_trip_revenue_bysubs']+trip_bymonth_cust['paid_trip_revenue_bycust']
trip_bymonth['paid_trip_revenue_bycust'] = trip_bymonth['paid_trip_monthly_revenue']-trip_bymonth['paid_trip_revenue_bysubs']

trip_bymonth_summ['All_pass_monthly_revenue'] = trip_bymonth_summ['pass_revenue_bysubs'] + trip_bymonth_summ['pass_revenue_bycust_1d'] + trip_bymonth_summ['pass_revenue_bycust_3d']

trip_bymonth['avg_reven_per_bike'] = np.around(5024220/trip_bymonth['no_of_bikes'], decimals=2)
trip_bymonth['avg_reven_per_trip'] = np.around(5024220/trip_bymonth['no_of_trips'], decimals=2)

print(trip_bymonth)
print('--------------------------------------------------------------------------')
print(trip_bymonth_summ)
print('--------------------------------------------------------------------------')

print("Total Q4 Revenue = " ,np.around(trip_bymonth_summ['All_pass_monthly_revenue'].sum() + trip_bymonth['paid_trip_monthly_revenue'].sum()))
print("Average monthly Revenue = " , np.around((trip_bymonth_summ['All_pass_monthly_revenue'].sum() + trip_bymonth['paid_trip_monthly_revenue'].sum())/3))


--------------------------------------------------------------------------
As per Citi Bike, avg distance traveled per second in miles is= 0.0020711111111111112
Total unique bikes used by subscribers in Q4= 13054
Total No of Trips by subscribers in Q4= 3726266
Each Subscriber pays per month= 14.95
--------------------------------------------------------------------------
           total_trip_time  no_of_bikes  no_of_trips  paidunits month  \
startdate                                                               
10              1355418246        11516      1663401   150177.0   OCT   
11               916593277        12773      1220354    85229.0   NOV   
12               602049732        11109       842511    57415.0   DEC   

           total_distance  avg_dis_per_trip  avg_time_per_trip  \
startdate                                                        
10             2807221.79              1.69              13.58   
11             1898366.52              1.56              12.52

In [13]:
# Few more cleanup and Descibe the dataframe.
df_sub['starthour'] = df_sub['starthour'].astype(int)
df_sub['endhour'] = df_sub['endhour'].astype(int)
df_sub['start_day'] = df_sub['start_day'].astype(int)

# Drop columns which are not needed to make df lighter 

df_sub.drop(['starttime', 'stoptime', 'start_name', 'end_name', 'birth year'], axis=1, inplace=True)
df_cust.drop(['starttime', 'stoptime', 'start_name', 'end_name', 'birth year'], axis=1, inplace=True)

print(df_sub.head())
df_sub.describe()


   trip_id  tripduration  start_id  start_lat  start_lon  end_id    end_lat  \
0        1           457       479  40.760193 -73.991255     478  40.760301   
2        2           761       504  40.732219 -73.981656     350  40.715595   
5        3           260      3443  40.761330 -73.979820     468  40.765265   
7        4           808      3305  40.781122 -73.949656     305  40.760958   
8        5          1143       284  40.739017 -74.002638     355  40.716021   

     end_lon  bikeid    usertype     ...      driver_age   startdate  \
0 -73.998842   30951  Subscriber     ...              33  2017-10-01   
2 -73.987030   28713  Subscriber     ...              26  2017-10-01   
5 -73.981923   27600  Subscriber     ...              28  2017-10-01   
7 -73.967245   17737  Subscriber     ...              25  2017-10-01   
8 -73.999744   20172  Subscriber     ...              26  2017-10-01   

  starthour     enddate endhour  start_day  end_day  free_trip  paidunits  \
0         0  20

,trip_id,tripduration,start_id,start_lat,start_lon,end_id,end_lat,end_lon,bikeid,gender,driver_age,starthour,endhour,start_day,end_day,free_trip,paidunits,paidamount
count,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06,3.726266e+06
mean,1.863134e+06,7.712979e+02,1.440701e+03,4.073716e+01,-7.398286e+01,1.428904e+03,4.073715e+01,-7.398373e+01,2.559676e+04,1.241310e+00,3.903543e+01,1.367407e+01,1.381377e+01,2.706733e+00,2.707514e+00,9.930877e-01,7.858296e-02,1.964574e-01
std,1.075680e+06,1.039112e+04,1.388033e+03,1.244255e-01,2.175515e-01,1.385444e+03,4.591808e-02,5.712936e-02,5.982522e+03,4.392722e-01,1.186143e+01,4.900341e+00,4.928896e+00,1.891782e+00,1.892211e+00,8.285230e-02,1.152036e+01,2.880089e+01
min,1.000000e+00,6.100000e+01,7.200000e+01,0.000000e+00,-7.403423e+01,7.200000e+01,0.000000e+00,-7.406692e+01,1.452900e+04,0.000000e+00,1.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.315672e+05,3.410000e+02,3.730000e+02,4.071912e+01,-7.399490e+01,3.680000e+02,4.071911e+01,-7.399530e+01,1.956700e+04,1.000000e+00,3.000000e+01,9.000000e+00,9.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,1.863134e+06,5.500000e+02,4.950000e+02,4.073902e+01,-7.398683e+01,4.920000e+02,4.073827e+01,-7.398700e+01,2.739700e+04,1.000000e+00,3.600000e+01,1.400000e+01,1.400000e+01,3.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,2.794700e+06,9.080000e+02,3.166000e+03,4.075641e+01,-7.397536e+01,3.164000e+03,4.075527e+01,-7.397575e+01,3.093300e+04,1.000000e+00,4.800000e+01,1.800000e+01,1.800000e+01,4.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,3.726266e+06,6.974419e+06,3.654000e+03,4.550585e+01,0.000000e+00,3.654000e+03,4.550636e+01,0.000000e+00,3.348100e+04,2.000000e+00,9.800000e+01,2.300000e+01,2.300000e+01,6.000000e+00,6.000000e+00,1.000000e+00,7.747000e+03,1.936750e+04


###### Calculated Average Monthly Revenue is  $5,024,220
###### --------------------------------------------------------------------------------------------------------------------

In [14]:
# Create another monthly summary DF with the encoded columns

df_new11 = df_sub.query("driver_age < 75 ")
df_new1 = df_new11.query("tripduration < 28800 ")

df_new1 = pd.get_dummies(df_new1, columns=['gender', 'start_day'], prefix=['gen', 'day'])


df_daily_trips_en = df_new1.groupby(['start_id','startdate'], as_index=False).agg({'tripduration': 'sum', 'bikeid':'nunique', 'trip_id':'count', 'paidunits':'sum', 'free_trip':'sum', 'driver_age':'median', 'paidamount':'sum',
                                                                              'gen_0':'sum','gen_1':'sum', 'gen_2':'sum', 'day_0':'sum', 'day_1':'sum', 'day_2':'sum', 'day_3':'sum', 
                                                                               'day_4':'sum', 'day_5':'sum', 'day_6':'sum' })
df_daily_trips_en = df_daily_trips_en.rename(index=str, columns={"tripduration": 'total_trip_time', "bikeid": "no_of_bikes", "trip_id": "no_of_trips", 'driver_age' :'median_user_age', 'gen_0' :'gen_othr', 'gen_1' :'gen_male', 'gen_2' :'gen_female'})
df_daily_trips_en['startdate'] = pd.to_datetime(df_daily_trips_en['startdate'])

df_near = df_near.rename(index=str, columns={"origin": 'start_id'})

df_daily_trips_en = df_daily_trips_en.merge(df_near, how='left', on='start_id')
df_daily_trips_en = df_daily_trips_en.merge(df_temp[['startdate', 'AVGT']], how='left', on='startdate')
df_daily_trips_en = df_daily_trips_en.merge(df_summ[['startdate', 'Members']], how='left', on='startdate')

df_daily_trips_en = df_daily_trips_en.rename(index=str, columns={"near_count": 'density_score' })
print(df_daily_trips_en.info())


<class 'pandas.core.frame.DataFrame'>
Index: 65729 entries, 0 to 65728
Data columns (total 22 columns):
start_id           65729 non-null int64
startdate          65729 non-null datetime64[ns]
total_trip_time    65729 non-null int64
no_of_bikes        65729 non-null int64
no_of_trips        65729 non-null int64
paidunits          65729 non-null float64
free_trip          65729 non-null int64
median_user_age    65729 non-null float64
paidamount         65729 non-null float64
gen_othr           65729 non-null uint8
gen_male           65729 non-null float64
gen_female         65729 non-null uint8
day_0              65729 non-null float64
day_1              65729 non-null float64
day_2              65729 non-null float64
day_3              65729 non-null float64
day_4              65729 non-null float64
day_5              65729 non-null float64
day_6              65729 non-null float64
density_score      65729 non-null int64
AVGT               65729 non-null float64
Members            6572

In [15]:
#Create a new summary dataframe for oct, daily summary per station
#add customer and subs together then 
#station id, date:- day, no of trips, total trip duration, avg trip duration, density score, freetrip flag, paid trips units,
#we will also drop the record with user age more than 75 considering it as outlier

df_new = df_sub.query("driver_age <= 75 ")
df_new1 = df_new.query("tripduration < 28800 ")

df_daily_trips = df_new1.groupby(['start_id','startdate'], as_index=False).agg({'tripduration': 'sum', 'bikeid':'nunique', 'trip_id':'count', 'paidunits':'sum', 'free_trip':'sum', 'start_day': 'max', 'driver_age':'median', 'paidamount':'sum', 'gender':'median' })
df_daily_trips = df_daily_trips.rename(index=str, columns={"tripduration": 'total_trip_time', "bikeid": "no_of_bikes", "trip_id": "no_of_trips", 'driver_age' :'median_user_age'})
df_daily_trips['startdate'] = pd.to_datetime(df_daily_trips['startdate'])

df_near = df_near.rename(index=str, columns={"origin": 'start_id'})

df_daily_trips = df_daily_trips.merge(df_near, how='left', on='start_id')
df_daily_trips = df_daily_trips.merge(df_temp[['startdate', 'AVGT']], how='left', on='startdate')
df_daily_trips = df_daily_trips.merge(df_summ[['startdate', 'Members']], how='left', on='startdate')

df_daily_trips = df_daily_trips.rename(index=str, columns={"near_count": 'density_score' })

df_daily_trips.head()



,start_id,startdate,total_trip_time,no_of_bikes,no_of_trips,paidunits,free_trip,start_day,median_user_age,paidamount,gender,density_score,AVGT,Members
0,72,2017-10-01,105707,98,100,1.0,99,6,35.0,2.5,1.0,473,61.0,246193
1,72,2017-10-02,120738,125,131,2.0,130,0,38.0,5.0,1.0,473,65.0,246402
2,72,2017-10-03,128572,135,140,3.0,139,1,36.0,7.5,1.0,473,65.0,246566
3,72,2017-10-04,146306,152,159,0.0,159,2,36.0,0.0,1.0,473,66.5,246742
4,72,2017-10-05,148665,160,162,8.0,160,3,36.5,20.0,1.0,473,75.0,246893


In [16]:
# Top Trip Routes

newdf = df_sub.groupby(['start_id','end_id'], as_index=False).agg({'tripduration': 'sum', 'trip_id':'count', 'paidunits':'sum' })
newdf = newdf.rename(index=str, columns={"trip_id": 'counts'})
newdf['paidratio'] = newdf['paidunits']/newdf['counts']

print(newdf.nlargest(20, 'counts'))
print("The Most busy route is between #432 and #3263")
print("The Most busy station is #519")

        start_id  end_id  tripduration  counts  paidunits  paidratio
63940        432    3263        525349    1999        3.0   0.001501
73775        460    3093        392324    1222       55.0   0.045008
64962        435     509        358050    1191        7.0   0.005877
93554        519     498        483084    1161        0.0   0.000000
93548        519     491        450729    1074        4.0   0.003724
89531        505     519        472928    1046        0.0   0.000000
177165      3430    3086        208018    1045        0.0   0.000000
119524      3107    3090        265962    1041       11.0   0.010567
121771      3118    3119        357119    1032        6.0   0.005814
117235      3093     460        323889    1024        3.0   0.002930
145380      3263     432        282424    1010        0.0   0.000000
93549        519     492        494381    1002        5.0   0.004990
160251      3351    3318        232651     996        1.0   0.001004
11627        239     270        29

In [17]:
# Top 3 Stations

df_top_station = df_sub[df_sub['start_id'].isin(['432','3263','519'])]
df_top_station1 = df_sub[df_sub['end_id'].isin(['432','3263','519'])]

print('% trip from top 3 station', 100*(len(df_top_station)/len(df_sub)))
print('% trip to top 3 station', 100*(len(df_top_station1)/len(df_sub)))

df_top_station.shape

% trip from top 3 station 2.018723301020378
% trip to top 3 station 2.0587097110082855


(75223, 21)

In [18]:
df_summ.to_csv('../data/df_summ_DV.csv', sep=',')
df_sub.to_csv('../data/df_sub_DV.csv', sep=',')
df_temp.to_csv('../data/df_temp_DV.csv', sep=',')
trip_bymonth.to_csv('../data/df_trip_bymonth_DV.csv', sep=',')
df_daily_trips.to_csv('../data/df_daily_trips_ML.csv', sep=',')

In [19]:
df_cust.to_csv('../data/df_cust_DV.csv', sep=',')
